In [2]:
import pandas as pd

df = pd.read_excel(r'C:\Users\jiamv\Downloads\Low_Conc_Experiments.xlsx')

X = df.iloc[:, [1] + list(range(3, 7))]

Y = df.iloc[:, list(range(1, 3))]


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)

Y_train_scaled = scaler_Y.fit_transform(Y_train)


In [5]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

svr = SVR(
    kernel='rbf',   # most common & powerful
    C=100,
    gamma='scale',
    epsilon=0.1
)

model = MultiOutputRegressor(svr)
model.fit(X_train_scaled, Y_train_scaled)


MultiOutputRegressor(estimator=SVR(C=100))

In [6]:
df.head()

date & time  Time in minutes   Grimm  Honeywell_01  Honeywell_03  \
0 2025-03-25 10:48:00                1  153.92    160.666667    288.666667   
1 2025-03-25 10:49:00                2  153.27    159.125000    283.500000   
2 2025-03-25 10:50:00                3  151.28    168.222222    279.555556   
3 2025-03-25 10:51:00                4  154.71    222.222222    274.666667   
4 2025-03-25 10:52:00                5  195.83    280.111111    266.888889   

   Honeywell_04  Plantower_01  Plantower_03  Plantower_04    SPS30_01  \
0    160.500000    146.444444    302.666667    163.875000  155.182222   
1    156.333333    142.375000    302.500000    163.555556  151.732500   
2    164.111111    152.555556    291.555556    182.555556  160.726667   
3    244.555556    238.555556    298.444444    282.000000  239.026667   
4    270.625000    308.555556    294.666667    322.625000  299.777778   

     SPS30_03    SPS30_04  
0  276.164444  139.057500  
1  271.147500  131.615556  
2  262.652222  142.931111  
3  259.520000  241.676667  
4  256.345556  271.020000

In [10]:
X_full_scaled = scaler_X.transform(X)
Y_full_pred_scaled = model.predict(X_full_scaled)
Y_full_pred = scaler_Y.inverse_transform(Y_full_pred_scaled)

Y_pred_df_full = pd.DataFrame(Y_full_pred, columns=Y.columns)
Y_pred_df_full['Time'] = df['Time in minutes']

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(Y, Y_full_pred)
mse = mean_squared_error(Y, Y_full_pred)
r2  = r2_score(Y, Y_full_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Support Vectors:", model.estimators_[0].support_vectors_)
print("Support Vector Indices:", model.estimators_[0].support_)
print("Dual Coefficients:", model.estimators_[0].dual_coef_)
print("Intercept:", model.estimators_[0].intercept_)
print("SVM Parameters:", model.estimators_[0].get_params())
print("Regularization (C):", model.estimators_[0].C)
print("Epsilon:", model.estimators_[0].epsilon)
print("Gamma (smoothing):", model.estimators_[0].gamma)

Mean Absolute Error: 3.137503806815861
Mean Squared Error: 12.924805981352538
R-squared: 0.9960504568265092
Support Vectors: [[-1.66225996  2.29357734  2.02873602  2.22743214  1.89094698]
 [-1.09348595  1.42786894  1.66025819  1.31715166  1.58038018]
 [-1.74351339  0.79496778  2.27777621  0.88412042  0.3010629 ]
 [-1.27630617  1.57389624  1.7860489   1.61803723  1.47113694]
 [ 0.87690971 -0.85228293 -0.83141431 -0.84800458 -0.85811406]
 [ 1.70975737 -0.86814031 -0.81743756 -0.86088031 -0.90742335]]
Support Vector Indices: [ 29  30  37  47 115 129]
Dual Coefficients: [[-1.43282654  0.45178858 -1.33297362  0.4640978  -1.48384337  3.33375715]]
Intercept: [-0.4308429]
SVM Parameters: {'C': 100, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Regularization (C): 100
Epsilon: 0.1
Gamma (smoothing): scale


In [ ]:
X_synthetic = X_train_scaled
Y_svm_pred = model.predict(X_synthetic)

from sklearn.ensemble import RandomForestRegressor
rf_student = RandomForestRegressor(n_estimators=8, max_depth=3)
rf_student.fit(X_synthetic, Y_svm_pred)

from micromlgen import port
with open("SVM_distilled.h", "w") as f:
    f.write(port(rf_student))